Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.
import datetime as dt
import numpy as np 
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".
start = dt.datetime(2020,9,22)
end = dt.datetime.now()

stock_symbol = ['AVAX-USD']

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.
deletd_column =[]
df = pdr.get_data_yahoo(stock_symbol, start, end).dropna()
df
scaler = StandardScaler()

scaled_df = pd.DataFrame(
    scaler.fit_transform(df), 
    columns=df.columns,
    index=df.index)
#df1 = df[["Open","Adj Close"]]

#df = #TODO
#df.head()
#df.shape
#df1.columns
#df["Open"]
scaled_df= scaled_df.drop("Close",axis="columns")
#df1=df
#df1 = df1[["Open","Adj Close"]]
scaled_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Adj Close,Volume
Date,,,,,
2020-09-22,-0.948016,-0.682762,-0.966446,-0.936187,-0.181802
2020-09-23,-0.936274,-0.954487,-1.000378,-1.016645,-0.483813
2020-09-24,-1.017556,-0.976769,-0.999961,-0.967014,-0.534519
2020-09-25,-0.967087,-0.970847,-0.960935,-0.960762,-0.609619
2020-09-26,-0.960422,-0.977903,-0.953540,-0.968761,-0.663657
...,...,...,...,...,...
2021-11-06,2.342108,2.301463,2.453179,2.514093,0.186426
2021-11-07,2.556161,2.752222,2.734153,2.937810,1.728250
2021-11-08,2.978646,3.110671,3.013578,3.101250,2.898848


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

scaled_df['Close_15_Rolling'] = scaled_df['Adj Close'].rolling(16).mean() #start from zero ?

scaled_df['Open_1_Change'] = scaled_df['Open'].diff()

scaled_df['Adj Close_Change'] = scaled_df['Adj Close'].diff(-1)

scaled_df['dif_high_low']= scaled_df['High']-scaled_df['Low']

scaled_df['Volume_Change'] = scaled_df['Volume'].diff()



scaled_df=scaled_df.dropna()
X=scaled_df
X

,Open,High,Low,Adj Close,Volume,Close_15_Rolling,Open_1_Change,Adj Close_Change,dif_high_low,Volume_Change
Date,,,,,,,,,,
2020-10-11,-0.993792,-1.007056,-0.974173,-0.989679,-0.693195,-0.985104,-0.010421,-0.002545,-0.032883,-0.005702
2020-10-14,-0.971886,-0.992541,-0.971588,-0.987134,-0.695876,-0.988288,0.021905,0.001066,-0.020953,-0.002681
2020-10-15,-0.987597,-1.004612,-0.971901,-0.988200,-0.701432,-0.986510,-0.015711,0.004081,-0.032711,-0.005556
2020-10-16,-0.988787,-1.007193,-0.978365,-0.992281,-0.696271,-0.988089,-0.001190,-0.007306,-0.028829,0.005161
2020-10-17,-0.993027,-1.005390,-0.973880,-0.984975,-0.703068,-0.989603,-0.004240,0.003755,-0.031510,-0.006797
...,...,...,...,...,...,...,...,...,...,...
2021-11-05,2.567634,2.407526,2.450127,2.311143,0.382520,1.972842,0.164718,-0.202950,-0.042600,-1.009687
2021-11-06,2.342108,2.301463,2.453179,2.514093,0.186426,2.023854,-0.225526,-0.423717,-0.151716,-0.196094
2021-11-07,2.556161,2.752222,2.734153,2.937810,1.728250,2.089178,0.214053,-0.163440,0.018069,1.541825


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.
#np.where(a < 5, a, 10*a)
y = X["Adj Close_Change"]
y = np.where(y<0,-1,1)
y.shape
X.shape
X=X.drop("Adj Close_Change",axis="columns")
X

,Open,High,Low,Adj Close,Volume,Close_15_Rolling,Open_1_Change,dif_high_low,Volume_Change
Date,,,,,,,,,
2020-10-11,-0.993792,-1.007056,-0.974173,-0.989679,-0.693195,-0.985104,-0.010421,-0.032883,-0.005702
2020-10-14,-0.971886,-0.992541,-0.971588,-0.987134,-0.695876,-0.988288,0.021905,-0.020953,-0.002681
2020-10-15,-0.987597,-1.004612,-0.971901,-0.988200,-0.701432,-0.986510,-0.015711,-0.032711,-0.005556
2020-10-16,-0.988787,-1.007193,-0.978365,-0.992281,-0.696271,-0.988089,-0.001190,-0.028829,0.005161
2020-10-17,-0.993027,-1.005390,-0.973880,-0.984975,-0.703068,-0.989603,-0.004240,-0.031510,-0.006797
...,...,...,...,...,...,...,...,...,...
2021-11-05,2.567634,2.407526,2.450127,2.311143,0.382520,1.972842,0.164718,-0.042600,-1.009687
2021-11-06,2.342108,2.301463,2.453179,2.514093,0.186426,2.023854,-0.225526,-0.151716,-0.196094
2021-11-07,2.556161,2.752222,2.734153,2.937810,1.728250,2.089178,0.214053,0.018069,1.541825


In [6]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

#y_data = X["Adj Close_Change"] 
#x_data = X.drop("Adj Close_Change", axis = 1)


#index = int(0.7*len(X))
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.30, random_state=0)

y_test.shape

(118,)

In [7]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.
logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)


#logistic = LogisticRegression()
#logistic = #TODO

In [8]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[0.40331745771701266]
1,High,[0.04655257077811527]
2,Low,[-0.11414642622698831]
3,Adj Close,[-0.10244140644808425]
4,Volume,[-0.16245475887390565]
5,Close_15_Rolling,[-0.12390726577567451]
6,Open_1_Change,[-0.14413550952684148]
7,dif_high_low,[0.16069899700510681]
8,Volume_Change,[0.07059143228519847]


In [9]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)
predictions

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1])

In [10]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)
predictions

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,
       -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1])

In [11]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.
from sklearn.metrics import accuracy_score 
accuracy = accuracy_score(y_test,predictions)
accuracy

0.5169491525423728

In [12]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val = cross_val_score(logistic,X, y, cv=5)
cross_val

array([0.53164557, 0.48101266, 0.44303797, 0.42307692, 0.3974359 ])